In [8]:
from math import exp, sqrt
from random import randrange,choice,random
import os
#import pylab
import numpy as np
#from scipy.optimize import curve_fit
from bisect import bisect_right

In [9]:
def correlator(ising):
    L=len(ising)
    #med=sum(ising)/float(L)
    Pm=ising.count(+1)/float(L)
    Mm=ising.count(-1)/float(L)
    Glob=Pm**2+Mm**2
    #print Glob
    correlations=[]

    cordist=np.logspace(3, np.log2(L/2),num=200,base=2, endpoint=True)
    cordist=map(int,cordist)
    cordist=list(set(cordist))
    cordist.sort()
    #print len(cordist)
    for dist in cordist:
        i=0
        positivCorr=0
        Num=float(L-dist)
        while (i<Num):
            if ising[i] == ising[i+dist]:
                positivCorr +=1
            i+=1
        connCorr=positivCorr/float(Num)
        #print str(connCorr-Glob)
        correlations.append(connCorr-Glob)
    return correlations

In [10]:
def funcex(x, A, b):
    return A*numpy.exp(-b*x)
def funcpl(x, B,a):
    return B*x**a

In [11]:
def find_gt(a, x):
    'Find leftmost value greater than x'
    i = bisect_right(a, x)
    if i != len(a):
        return a[i]
    else: return 1

In [12]:
def init_ising_lattice(n,cool):
    if cool == False:
        lattice = [0]*n
        options = [-1,1]
        for i in range(n):
            lattice[i] = choice(options)
    else:
        lattice=[1]*n
        """out_f = open("./toComplete-mfOFF.is","r")
        file=out_f.read()
        latticeT=list(file)
        lattice=[1 if x=='+' else -1 for x in latticeT]    
        out_f.close()"""
    return lattice

In [13]:
#crea file sequenza
def pil_image(lattice,beta,alpha):
   
    n= len(lattice)
    
    isingRG=''
    for i in range(n):
        if lattice[i]>0: isingRG=isingRG + '+'
        else: isingRG=isingRG + '-'
        
    directory='./param-beta'+str(beta)+'alpha'+str(alpha)

    if not os.path.isdir(directory):
        os.mkdir(directory)
    out_file = open(directory + "/isingRG.is","w")

    out_file.write (isingRG)


    out_file.close()
    return

In [14]:
  
def cum_prob(summ):
    return 1-np.exp(-2*summ)

def tool(beta,alpha,d):
    return beta*(d**alpha)

def lookUpTable(L,beta,alpha):
    #define lookup table
    lUpTab=[]
    summa=0
    for dist in range(1,L+1):
        summa+=tool(beta,alpha,dist)
        Cprob=cum_prob(summa)
        lUpTab.append(Cprob)
    return lUpTab

def cluster_for_a_node(n,i,lookup,ising,tempCluster):
    distanceToEnd=n-i
    distanceToBeg=i
    rescaling=1
    Cd=0
    distance=0
    while (distanceToEnd>0):
        g=random()*rescaling+Cd
            #if g >= lookup[-1]: break
        
        Cd=find_gt(lookup,g)
        #print Cd
        distance=bisect_right(lookup, g)
            #print distance
        if i+distance >= n: break
        if ising[i] == ising[i+distance] and i+distance not in tempCluster:
            tempCluster.append(i+distance)
            distanceToEnd=n-i-distance
            #outOfCluster.remove(i+distance)
            #search.remove(i+distance)
        rescaling=1-Cd
        
    
    distance=0
    rescaling=1
    Cd=0
    while (distanceToBeg>0):
        g=random()*rescaling+Cd
            #if g >= lookup[-1]: break
        Cd=find_gt(lookup,g)
        distance=bisect_right(lookup, g)
        if i-distance <= 0: break
        if ising[i] == ising[i-distance]and i-distance not in tempCluster:
            tempCluster.append(i-distance)
            distanceToEnd=i-distance
                #outOfCluster.remove(i-distance)
                #search.remove(i-distance)
        rescaling=(1-Cd)
        
    return tempCluster




def clustering(ising,lookup,n):
    
    #outOfCluster=range(n)
    #search=set(outOfCluster)
    clusters=[]
    #while(len(outOfCluster)>(0.9*n)):
    i=choice(range(n))
    #print i
    tempCluster=[i]
    #tempCluster.append(i)
    #outOfCluster.remove(i)
    #    search.remove(i)
        #single cluster
    new=tempCluster
    while(len(new)>0):
        for item in new:
            tempCluster=cluster_for_a_node(n,item,lookup,ising,tempCluster)
        #print "lung clust"+str(len(tempCluster))
        temp=[item for item in tempCluster if item not in new]
        new=temp
            #if len(outOfCluster)==0:break
            
       
    #print "finito cluster"    
    clusters.append(tempCluster)
        
    
    #clusters needs to be a list of lists
    #print len(clusters)
    #print len(clusters[1])
    return clusters

def clusterSwitch(ising,clusters,h_ex):
    #options=[-1,1]  #1/2 probability to switch  
    h=h_ex
    L=len(ising)
                                   
    for cluster in clusters:
        clusterSize=len(cluster)
        clusterImp=clusterSize/float(L)
        #if clusterSize >0.7*L: print clusterSize
        threshold=np.exp(h*clusterImp)/(np.exp(h*clusterImp)+np.exp(-h*clusterImp))
        if random()<=threshold:
            newSign=-1
        else:
            newSign=1
        toSwitchList=cluster
        
        #newSign=choice(options)  #1/2 probability to switch
        for node in toSwitchList:
            ising[node]=newSign
            
    #modify the lattice by randomly switching clusters
    return ising

def statistics(ising):
    #do the math
    Magnet=sum(ising)/float(len(ising))
    return Magnet
  

In [19]:
def main():
    import sys,getopt
    #opts,args = getopt.getopt(sys.argv[1:],'n:s:b:a:c:h:')
    n = 100
    nsteps = 10**5
    beta=0.94
    alpha=-1.875
    h=0.
    cool=False
    #for key,val in opts:
    #    if key == '-n': n = int(val)
    #    elif key == '-s': nsteps = int(val)
    #    elif key == '-b': beta = float(val)
    #    elif key == '-a': alpha= float(val)
    #    elif key == '-c': cool = bool(val)
    #    elif key == '-h': h = float(val)
    
    tab=lookUpTable(n,beta,alpha)
    #print tab
    if (1-tab[-1])<=10**(-9): print 'cacca'
    #print tab[-1]
    magnetiz=[]
    term=[]
    ising=init_ising_lattice(n,cool)
    correlations=[]
    for i in range(nsteps):
        #termalizzazione
        if i%10==0:
            term.append(statistics(ising))
        if i%5000==0:
            
            out_f = open("./whereAmI"+str(beta),"w")
            out_f.write ("son qui "+str(i)+ " su "+str(nsteps)+'\n')
            out_f.close()
    
    
       
        if i > nsteps-10000:
            if i%100==0:
                correlations.append(correlator(ising))
                magnetiz.append(statistics(ising))
        if i%150000**1==0: print str(i/15000**1)+'%'
        clu=clustering(ising,tab,n)
        ising=clusterSwitch(ising,clu,h)
    #print clu

    Mmedio=np.mean(np.absolute(magnetiz))
    correlationsTranspose=map(list,zip(*correlations))
    correlationsMean=[np.mean(item) for item in correlationsTranspose]
    correlationsSTD=[np.var(item) for item in correlationsTranspose]
    #cor.fitting(10,correlationsMean,correlationsSTD,n)
    pil_image(ising,beta,alpha)
    stats=[len(item) for item in clu]
    #pylab.hist(stats,40)
    #pylab.show()
    #pylab.plot(np.absolute(magnetiz))
    #pylab.show()


    cordist=np.logspace(3, np.log2(n/2),num=200,base=2, endpoint=True)
    cordist=map(int,cordist)
    cordist=list(set(cordist))
    cordist.sort()
    print 'dist'
    print len(cordist)
    print len(correlationsMean)
    stringf='./length'+str(n)+'param-beta'+str(beta)+'alpha'+str(alpha)+'h'+str(h)+'plaw.dat'
    out_corr = open(stringf,"w")
   
    out_corr.write ("#distances correlations errors     Mean Magnetization =" +str(Mmedio)+" \n")
    for i,num in enumerate(cordist):
        out_corr.write (str(cordist[i])+" "+str( correlationsMean[i])+'  '+str(correlationsSTD[i])+"\n")

    out_corr.close()
    
    #controllo termalizzazione
    out_term=open(stringf+'term.txt','w')
    
    for m in term:
        out_term.write(str(m)+'\n')
    out_term.close()
    
    bins=[2**i for i in range(4,200)]
#print bins
    v=[]
    for item in bins:
        v.append(term[:item])
        term=[x for x in term if x not in term[:item]]
        if not term: break
    Mean=[np.mean(item) for item in v]
    Std=[np.std(item) for item in v]

    diff=np.array(Mean[:-1])-np.array(Mean[1:])
    diff=map(np.absolute,diff)
    StdN=np.array(Std[1:])

    Check=diff-StdN
    print Check


    index=next(x[0] for x in enumerate(Check) if x[1] <= 0)
    Eq=bins[index+1]

    print 'Termalizzo allo step ',Eq
    
if __name__ == '__main__': main()

0%
dist
42
42
[-0.49948088 -0.52207341 -0.38440651 -0.55378023]
Termalizzo allo step  32
